In [1]:
import pandas as pd
import numpy as np
from apyori import apriori

In [2]:
article = pd.read_csv("articles.csv")
customer = pd.read_csv("customers.csv")
sample_submission = pd.read_csv("sample_submission.csv")
transaction_train = pd.read_csv("transactions_train.csv")

In [3]:
transaction_train.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [4]:
transaction_train.describe()

,article_id,price,sales_channel_id
count,3.178832e+07,3.178832e+07,3.178832e+07
mean,6.962272e+08,2.782927e-02,1.704028e+00
std,1.334480e+08,1.918113e-02,4.564786e-01
min,1.087750e+08,1.694915e-05,1.000000e+00
25%,6.328030e+08,1.581356e-02,1.000000e+00
50%,7.145820e+08,2.540678e-02,2.000000e+00
75%,7.865240e+08,3.388136e-02,2.000000e+00
max,9.562170e+08,5.915254e-01,2.000000e+00


In [5]:
transaction_train.isnull().sum()

t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
dtype: int64

In [6]:
customer.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [7]:
pd.options.mode.chained_assignment = None

ref: https://medium.com/@jihargifari/how-to-perform-market-basket-analysis-in-python-bd00b745b106

In [8]:
transaction_train = transaction_train[['t_dat','customer_id','article_id']]

In [9]:
print(min(transaction_train['t_dat']),max(transaction_train['t_dat']))

2018-09-20 2020-09-22


In [47]:
df = transaction_train[transaction_train['t_dat']>='2020-07-01']

In [48]:
# assume items bought by the same customer on the same day is on the same receipt (basket)
df['dummy_invoice']=df.groupby(['t_dat','customer_id']).ngroup()
df.reset_index()

,index,t_dat,customer_id,article_id,dummy_invoice
0,28401361,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,825577003,0
1,28401362,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,849487005,0
2,28401363,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,772659001,0
3,28401364,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,804732004,0
4,28401365,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,804732004,0
...,...,...,...,...,...
3386958,31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,1040900
3386959,31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,1040900
3386960,31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,1040901
3386961,31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,1040902


In [49]:
basket['no_unique_article'] = basket.groupby('dummy_invoice')['article_id'].transform('nunique')
basket.head(15)

,dummy_invoice,article_id,quantity,no_unique_article
0,0,777148006,1,4
1,0,835801001,1,4
2,0,865929003,1,4
3,0,923134005,1,4
4,1,863646004,1,1
5,2,523490007,1,6
6,2,574109042,1,6
7,2,797079002,1,6
8,2,816586006,1,6
9,2,855834006,1,6


In [50]:
basket_plus = basket[basket['no_unique_article']>1]
print(basket.shape,basket_plus.shape)

(723291, 4) (631053, 4)


In [61]:
# cast to str due to an error from frequent itemset mining later
basket_plus.article_id = [str(i) for i in basket_plus.article_id]

In [62]:
transactions = basket_plus.groupby('dummy_invoice')['article_id'].apply(list)
transactions

dummy_invoice
0              [777148006, 835801001, 865929003, 923134005]
2         [523490007, 574109042, 797079002, 816586006, 8...
3         [811899002, 827411001, 832330001, 882757004, 8...
5                                    [622958016, 805917001]
7                                    [751664001, 919365008]
                                ...                        
255893                               [740519002, 895730002]
255894    [881448001, 881448002, 919273001, 919273002, 9...
255895                               [918485001, 918522001]
255896    [456163087, 701472004, 791587021, 832505003, 8...
255897                               [891322004, 929511001]
Name: article_id, Length: 163663, dtype: object

In [63]:
# try sparse
oht_ary = te.fit(transactions).transform(transactions, sparse=True)
basket_encode = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
basket_encode

,108775044,110065011,111565001,111586001,111593001,111609001,120129001,120129014,123173001,126589007,...,948152002,949198001,949551001,949551002,949594001,952267001,952938001,953450001,953763001,956217002
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163658,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163659,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163660,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163661,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
# for ref only, this is not sparse matrix
# from mlxtend.preprocessing import TransactionEncoder

# te = TransactionEncoder()
# te_ary = te.fit(transactions).transform(transactions)
# basket_encode = pd.DataFrame(te_ary, columns=te.columns_)
# basket_encode

In [45]:
from mlxtend.frequent_patterns import apriori

In [64]:
frequent_itemsets = apriori(basket_encode, min_support=0.004,
                                 use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)

frequent_itemsets['length'] = frequent_itemsets_plus['itemsets'].apply(lambda x: len(x))

frequent_itemsets

,support,itemsets,length
0,0.008933,(751471001),1
1,0.008353,(915529003),1
2,0.008194,(918292001),1
3,0.007735,(924243001),1
4,0.007185,(751471043),1
5,0.007167,(706016001),1
6,0.007161,(918522001),1
7,0.006862,(448509014),1
8,0.006465,(915526001),1
9,0.006452,(865799006),1


In [54]:
from mlxtend.frequent_patterns import association_rules

In [ ]:
# calculate quantity of item on the same receipt
# basket = df.groupby(['dummy_invoice','article_id']).size()
# basket = pd.DataFrame(basket).reset_index()
# basket.rename(columns={basket.columns[2]: 'quantity'}, inplace = True)
# basket

In [ ]:
# have not started - experiment with https://www.kaggle.com/cdeotte/customers-who-bought-this-frequently-buy-this/notebook

In [18]:
vc = df.article_id.value_counts()

In [19]:
vc

706016001    50287
706016002    35043
372860001    31718
610776002    30199
759871002    26329
             ...  
254457030        1
581293001        1
868812001        1
867569001        1
551124007        1
Name: article_id, Length: 104547, dtype: int64

In [26]:
df.loc[df.article_id==534357002,'customer_id']

6612        23b1f7858432cffa3130687990c0a7fb09f73badfebf3c...
14982       4f44b9a8d6c12db5a703d22b0fa08d6d433bbeda020436...
18312       5fe3e8928770a90d287d620fade678325b6e2c04e04fd7...
18347       5ffd00f17e6effccd88b49c56c265c4d0150e4a4514e87...
26781       8c70d5ad7e345ed27a817d1baa393c72190d1e7c928ad2...
                                  ...                        
29577667    364cf7ff690d4f2190a8ed4354ee1ffd7b055d0e20f7ef...
29588874    74fc5dd0036a0d0db26379f13e9e8e760569c46d16abf7...
29594876    9563eda8d23992b79ef229562ea02f03085a9e2fb91642...
30646636    71e73b1fad436f5925e41529335b47226f524467426c95...
30893310    71e73b1fad436f5925e41529335b47226f524467426c95...
Name: customer_id, Length: 3257, dtype: object

In [28]:
for j,i in enumerate(vc.index.values[1000:1001]):
    print(j,i)
    USERS = df.loc[df.article_id==i.item(),'customer_id'].unique()
    vc2 = df.loc[(df.customer_id.isin(USERS))&(df.article_id!=i.item()),'article_id'].value_counts()
    print(vc2)

0 534357002
706016001    264
534357008    247
400319005    229
448509014    225
640021005    197
            ... 
683639006      1
894593004      1
665082021      1
815456006      1
507883018      1
Name: article_id, Length: 42775, dtype: int64
